In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q flwr["simulation"] tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.3

In [3]:
import flwr as fl
import tensorflow as tf
import numpy as np
from typing import List, Tuple
from flwr.common import Metrics
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [4]:
train_data = np.load("/content/drive/MyDrive/federated_learning_ecg/X_train.npy")
train_label = np.load("/content/drive/MyDrive/federated_learning_ecg/y_train.npy")
test_data = np.load("/content/drive/MyDrive/federated_learning_ecg/X_test.npy")
test_label = np.load("/content/drive/MyDrive/federated_learning_ecg/y_test.npy")

In [5]:
x_train = []
y_train = []
x_test = []
y_test = []
for i in range(6):
  x_train.append(train_data[train_data.shape[0]//6*i:train_data.shape[0]//6*(i+1)])
  y_train.append(train_label[train_label.shape[0]//6*i:train_label.shape[0]//6*(i+1)])
  x_test.append(test_data[test_data.shape[0]//6*i:test_data.shape[0]//6*(i+1)])
  y_test.append(test_label[test_label.shape[0]//6*i:test_label.shape[0]//6*(i+1)])

In [6]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, x_train_cid, y_train_cid, x_test_cid, y_test_cid) -> None:
        self.model = model
        self.x_train_cid, self.y_train_cid = x_train_cid, y_train_cid
        self.x_test_cid, self.y_test_cid = x_test_cid, y_test_cid

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train_cid, self.y_train_cid, epochs=5, batch_size=32, verbose=0)
        return self.model.get_weights(), len(self.x_train_cid), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, acc = self.model.evaluate(self.x_test_cid, self.y_test_cid, verbose=0)
        return loss, len(self.x_test_cid), {"accuracy": acc}

In [7]:
def client_fn(cid: str) -> fl.client.Client:

    model = Sequential([
      layers.Conv1D(16, 3, padding='same', activation='relu', input_shape=(180,1)),
      layers.MaxPool1D(),
      layers.Conv1D(32, 3, padding='same', activation='relu'),
      layers.MaxPool1D(),
      layers.Conv1D(64, 3, padding='same', activation='relu'),
      layers.MaxPool1D(),
      layers.Flatten(),
      layers.Dense(256, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(5 ,activation="softmax")
    ])
    model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy()])

    x_train_cid = x_train[int(cid)]
    y_train_cid = y_train[int(cid)]

    x_test_cid = x_test[int(cid)]
    y_test_cid = y_test[int(cid)]


    return FlowerClient(model, x_train_cid, y_train_cid, x_test_cid, y_test_cid)

In [8]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:

    print(metrics)
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]


    return {"accuracy": sum(accuracies) / sum(examples)}

In [9]:
#created for initial parameters
model = Sequential([
  layers.Conv1D(16, 3, padding='same', activation='relu', input_shape=(180,1)),
  layers.MaxPool1D(),
  layers.Conv1D(32, 3, padding='same', activation='relu'),
  layers.MaxPool1D(),
  layers.Conv1D(64, 3, padding='same', activation='relu'),
  layers.MaxPool1D(),
  layers.Flatten(),
  layers.Dense(256, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(5 ,activation="softmax")
])

In [10]:
strategy = fl.server.strategy.FedYogi(
    fraction_fit=1.0,
    fraction_evaluate=0.5,
    min_fit_clients=6,
    min_evaluate_clients=3,
    min_available_clients=6,
    initial_parameters=fl.common.ndarrays_to_parameters(model.get_weights()),
    evaluate_metrics_aggregation_fn=weighted_average,
)

client_resources = None


fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=6,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-06-16 19:26:40,740 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-06-16 19:26:45,557	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-16 19:26:49,455 | app.py:180 | Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'CPU': 2.0, 'memory': 7809483572.0, 'object_store_memory': 3904741785.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'CPU': 2.0, 'memory': 7809483572.0, 'object_store_memory': 3904741785.0}
INFO flwr 2023-06-16 19:26:49,464 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-16 19:26:49,467 | server.py:269 | Using initial parameters provided by strategy
INFO:flwr:Using initial parameters provided by strategy
INFO flwr 2023-06-16 19:26:49,471 | server.py:88 | Evaluating initial para

[(5473, {'accuracy': 0.8452402949333191}), (5473, {'accuracy': 0.8454229831695557}), (5473, {'accuracy': 0.8516352772712708})]


DEBUG flwr 2023-06-16 19:33:32,433 | server.py:232 | fit_round 2 received 6 results and 0 failures
DEBUG:flwr:fit_round 2 received 6 results and 0 failures
DEBUG flwr 2023-06-16 19:33:32,483 | server.py:168 | evaluate_round 2: strategy sampled 3 clients (out of 6)
DEBUG:flwr:evaluate_round 2: strategy sampled 3 clients (out of 6)
DEBUG flwr 2023-06-16 19:33:35,454 | server.py:182 | evaluate_round 2 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 2 received 3 results and 0 failures
DEBUG flwr 2023-06-16 19:33:35,458 | server.py:218 | fit_round 3: strategy sampled 6 clients (out of 6)
DEBUG:flwr:fit_round 3: strategy sampled 6 clients (out of 6)


[(5473, {'accuracy': 0.8847067356109619}), (5473, {'accuracy': 0.8841586112976074}), (5473, {'accuracy': 0.8868993520736694})]


DEBUG flwr 2023-06-16 19:36:48,370 | server.py:232 | fit_round 3 received 6 results and 0 failures
DEBUG:flwr:fit_round 3 received 6 results and 0 failures
DEBUG flwr 2023-06-16 19:36:48,423 | server.py:168 | evaluate_round 3: strategy sampled 3 clients (out of 6)
DEBUG:flwr:evaluate_round 3: strategy sampled 3 clients (out of 6)
DEBUG flwr 2023-06-16 19:36:52,324 | server.py:182 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
DEBUG flwr 2023-06-16 19:36:52,328 | server.py:218 | fit_round 4: strategy sampled 6 clients (out of 6)
DEBUG:flwr:fit_round 4: strategy sampled 6 clients (out of 6)


...

DEBUG flwr 2023-06-16 19:56:35,410 | server.py:232 | fit_round 9 received 6 results and 0 failures
DEBUG:flwr:fit_round 9 received 6 results and 0 failures
DEBUG flwr 2023-06-16 19:56:35,462 | server.py:168 | evaluate_round 9: strategy sampled 3 clients (out of 6)
DEBUG:flwr:evaluate_round 9: strategy sampled 3 clients (out of 6)
DEBUG flwr 2023-06-16 19:56:38,373 | server.py:182 | evaluate_round 9 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 9 received 3 results and 0 failures
DEBUG flwr 2023-06-16 19:56:38,378 | server.py:218 | fit_round 10: strategy sampled 6 clients (out of 6)
DEBUG:flwr:fit_round 10: strategy sampled 6 clients (out of 6)


[(5473, {'accuracy': 0.9714964628219604}), (5473, {'accuracy': 0.9720445871353149}), (5473, {'accuracy': 0.9694865942001343})]


DEBUG flwr 2023-06-16 19:59:52,273 | server.py:232 | fit_round 10 received 6 results and 0 failures
DEBUG:flwr:fit_round 10 received 6 results and 0 failures
DEBUG flwr 2023-06-16 19:59:52,344 | server.py:168 | evaluate_round 10: strategy sampled 3 clients (out of 6)
DEBUG:flwr:evaluate_round 10: strategy sampled 3 clients (out of 6)
DEBUG flwr 2023-06-16 19:59:57,670 | server.py:182 | evaluate_round 10 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 3 results and 0 failures
INFO flwr 2023-06-16 19:59:57,673 | server.py:147 | FL finished in 1988.157069713
INFO:flwr:FL finished in 1988.157069713
INFO flwr 2023-06-16 19:59:57,681 | app.py:218 | app_fit: losses_distributed [(1, 1.3615020116170247), (2, 1.1695900758107503), (3, 0.7998127142588297), (4, 0.37349948287010193), (5, 0.21450870235761008), (6, 0.16669270396232605), (7, 0.1773796280225118), (8, 0.20137900610764822), (9, 0.22182663778463999), (10, 0.23625731964906058)]
INFO:flwr:app_fit: losses_distributed [

[(5473, {'accuracy': 0.9640051126480103}), (5473, {'accuracy': 0.966197669506073}), (5473, {'accuracy': 0.9676594138145447})]


History (loss, distributed):
	round 1: 1.3615020116170247
	round 2: 1.1695900758107503
	round 3: 0.7998127142588297
	round 4: 0.37349948287010193
	round 5: 0.21450870235761008
	round 6: 0.16669270396232605
	round 7: 0.1773796280225118
	round 8: 0.20137900610764822
	round 9: 0.22182663778463999
	round 10: 0.23625731964906058
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.8474328517913818), (2, 0.8852548996607462), (3, 0.8992021481196085), (4, 0.9130885004997253), (5, 0.9387295246124268), (6, 0.9593154390652975), (7, 0.968572994073232), (8, 0.9728363553682963), (9, 0.9710092147191366), (10, 0.965954065322876)]}